In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


# （2022年1月19日～）最適モデルをまとめた表を作成する処理の確認

`returnDictAboutMAPETable()` の問題に伴い、最適モデルの組み合わせをまとめた表の結果が怪しい。修正する。

# （2022年1月19日～）MAPEをまとめた表を作成する関数の修正もしくは別バージョンの作成

`returnDictAboutMAPETable()`は生データを取得する機能とMAPE表を作成する機能がまとまっている。

* 生データを取得する機能を持つ関数の作成
    * 引数
        * ベンチマーク名
        * 問題サイズのリスト
        * コア数のリスト
        * 問題サイズごとの初期パラメータの付与の有無
* 生データからMAPEを作成する関数の作成
    * 引数
        * 既存の`returnDictAboutMAPETable()`における引数で生データの取得のみに必要な引数は除く

In [2]:
# 引数に該当する生データを取得する関数
# 引数
# benchmark_names：ベンチマーク名のリスト
# classes：問題サイズのリスト
# processes：コア数のリスト
# csv_dir_path：CSVファイルを格納しているディレクトリのパス


def return_rawDF_with_init_param(
    benchmark_name="", classes=[], processes=[], csv_dir_path="./csv_files/"
):
    if benchmark_name == "":
        raise ExceptionInResearchLib("return_rawDF_with_init_param()の引数benchmark_nameが空文字列です")
    if classes == []:
        raise ExceptionInResearchLib("return_rawDF_with_init_param()の引数classesが空リストです")
    if processes == []:
        raise ExceptionInResearchLib("return_rawDF_with_init_param()の引数processesが空リストです")
    if os.path.exists(csv_dir_path) == False:
        raise ExceptionInResearchLib("return_rawDF_with_init_param()の引数csv_dir_pathに該当するディレクトリが存在しません")
    
    rawDF = returnCollectedExistingData(
        benchmarkNames=[benchmark_name],
        classes=classes,
        processes=processes,
        csvDirPath=csv_dir_path,
    )
    print(rawDF)
    return 0


def test_return_rawDF_with_init_param():
    classes = ["A", "B", "C", "D"]
    processes = [2, 4, 6, 8, 16, 32]
    # TODO:ライブラリノートへ移行時に "../csv_files/"
    csv_dir_path = "./csv_files/"
    # ベンチマークCGで実施
    benchmark_name = "cg"
    # ベンチマークCGの生データをテスト関数を用いて取得
    rawDF = return_rawDF_with_init_param(
        benchmark_name=benchmark_name,
        classes=classes,
        processes=processes,
        csv_dir_path=csv_dir_path,
    )
    # 初期変数として存在すべき値をリスト化
    init_param_names = ["na", "nonzer", "niter", "shift"]
    # 取得したDFに初期化変数があることを確認
    column_names_from_DF = rawDF.columns.tolist()
    for init_param_name in init_param_names:
        assert init_param_name in column_names_from_DF

    # ベンチマークISで実施
    benchmark_name = "is"
    # ベンチマークISの生データをテスト関数を用いて取得
    rawDF = return_rawDF_with_init_param(
        benchmark_name=benchmark_name,
        classes=classes,
        processes=processes,
        csv_dir_path=csv_dir_path,
    )
    # 初期変数として存在すべき値をリスト化
    init_param_names = [
        "TOTAL_KEYS_LOG_2",
        "MAX_KEY_LOG_2",
        "NUM_BUCKETS_LOG_2",
        "MIN_PROCS",
        "ONE",
    ]
    # 取得したDFに初期化変数があることを確認
    column_names_from_DF = rawDF.columns.tolist()
    for init_param_name in init_param_names:
        assert init_param_name in column_names_from_DF

In [3]:
test_return_rawDF_with_init_param()

            functionName  functionCallNum benchmarkName benchmarkClass  \
0       .TAU_application              1.0            cg              A   
1                     CG              1.0            cg              A   
2         MPI_Finalize()              1.0            cg              A   
3         INITIALIZE_MPI              1.0            cg              A   
4             MPI_Init()              1.0            cg              A   
..                   ...              ...           ...            ...   
17          MPI_Reduce()              1.0            cg              D   
18  SETUP_SUBMATRIX_INFO              1.0            cg              D   
19       MPI_Comm_size()              1.0            cg              D   
20       MPI_Comm_rank()              1.0            cg              D   
21       SETUP_PROC_INFO              1.0            cg              D   

    process  
0         2  
1         2  
2         2  
3         2  
4         2  
..      ...  
17       32  

AttributeError: 'int' object has no attribute 'columns'

# （2022年1月15日～2022年1月18日）組み合わせモデルの検証

問題サイズ固定時とコア数固定時での最適モデルから予測される組み合わせモデルが、実際に選択される組み合わせモデルと異なる。したがって、その異なっている原因がプログラムの実装ミスに由来するものなのかどうかを検証する。

やること（TODO）

1. ✅問題サイズ固定時とコア数固定時とでは異なるモデルを算出する関数をDFとして抽出
2. ✅各関数でモデルを構築
3. ✅出力した表の列名を英語から日本語にする
4. ✅現行で最適となっているモデルと最適となるはずの組み合わせモデルの違いを探す
    1. returnDictAboutMAPETable()に「ベンチマークプログラムごとに異なる問題サイズ由来の初期変数を追加する処理が含まれていない」問題があったのでそれをやらないといけない

In [ ]:
resultsDict = {}

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128]
benchmark_name = "cg"

# ベンチマークプログラム名のリスト
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIp": "反比例",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLog": "対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月15日"

target_function_names = ["CONJ_GRAD", "ICNVRT", "SPRNVC", "VECSET"]

expVar = returnExplanatoryVariablesList(
    benchmarkName=benchmark_name,
    classes=classes,
    processes=processes,
    csvDirPath=csvDirPath,
    baseExpVar=True,
    initExpVar=True,
)

rawDF = returnCollectedExistingData(
    benchmarkNames=[benchmark_name],
    classes=classes,
    processes=processes,
    csvDirPath=csvDirPath,
)

target_functions_rawDF = rawDF[
    (rawDF["functionName"] == target_function_names[0])
    | (rawDF["functionName"] == target_function_names[1])
    | (rawDF["functionName"] == target_function_names[2])
    | (rawDF["functionName"] == target_function_names[3])
]

# 説明変数用に問題サイズを数値化した列を追加
strListProblemSize = target_functions_rawDF["benchmarkClass"].tolist()
intListProblemSize = convertBenchmarkClasses_problemSizeInNPB(
    inputList=strListProblemSize
)
target_functions_rawDF["intBenchmarkClass"] = intListProblemSize
# 説明変数用に問題サイズ由来のほかの数値を保持する列を追加
target_functions_rawDF = addInitDataToRawDF(target_functions_rawDF)

# 目的変数の列名を保持したリスト
res_var = ["functionCallNum"]
# 説明変数の列名を保持したリスト
exp_var = target_functions_rawDF.columns.tolist()
exp_var.remove("functionName")
exp_var.remove("benchmarkName")
exp_var.remove("benchmarkClass")
print(exp_var)

# 結果を格納するためのDFを作成
resultDF = pd.DataFrame(columns=["functionName"] + model_names_list)
print(resultDF)

for target_function_name in target_function_names:
    # 関数ごとに生データを取得
    target_function_rawDF = target_functions_rawDF[
        target_functions_rawDF["functionName"] == target_function_name
    ]

    # モデルの構築
    models = Models(
        inputDF=target_function_rawDF,
        expVarColNames=exp_var,
        resVarColNames=res_var,
        targetDF=None,
        modelNames=model_names_list,
    )
    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    # MAPEの算出
    dictCalcedMAPE = models.returnCalculatedMAPE()

    # 算出されたMAPEの数値を小数第一位までにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10

    # 関数ごとの結果に格納
    dict_for_series = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = target_function_name

    series = pd.Series(dict_for_series)
    resultDF = resultDF.append(series, ignore_index=True)

resultDF = resultDF.rename(columns=dictColumnNamesJP)

print(
    resultDF.to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}における特定の関数でのMAPE",
        index=False,
        label=f"{date}bestModelCombinationIn{benchmark_name.upper()}inTargetFunctions",
    )
)

# df = returnDictAboutMAPETable(
#     benchmarkName=benchmark_name,
#     classes=classes,
#     targetClass=None,
#     processes=processes,
#     targetProcess=None,
#     expVar=expVar,
#     csvDirPath=csvDirPath,
#         modelNames=model_names_list,
# )
# df

# （～2022年1月14日）組み合わせモデルにおいて最適なモデルを出力する

| 関数名    | コア数固定での最適モデル | 問題サイズ固定での最適モデル | 
| --------- | ------------------------ | ---------------------------- | 
| func_name | model_name               | model_name                   | 

上記のような表を作成する。

In [ ]:
resultsList = []

# コア数を128に固定し、問題サイズを振る
classes_fix_core = ["A", "B", "C", "D", "E", "F"]
processes_fix_core = [128]
# 問題サイズをBに固定し、コア数を振る
classes_fix_class = ["B"]
processes_fix_class = [2, 4, 8, 16, 32, 64, 128]
# ベンチマークプログラム名のリスト
benchmark_names = ["cg"]
# benchmark_names = ["cg", "ep", "ft", "is", "lu", "mg"]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

dict_column_fix_core = {"最適モデル": "最適モデル（コア数固定）"}
dict_column_fix_class = {"最適モデル": "最適モデル（問題サイズ固定）"}

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月10日"

# 方針

1. ✅「最適モデル」列を追加する関数を作成
2. ✅作成した関数をループ内で使用
3. ✅ループ内で、「コア数を128に固定し、問題サイズを振る」と「問題サイズをBに固定し、コア数を振る」
4. 2つの実験結果を結合させる
    1. ✅それぞれの実験結果から列「最適モデル」を取得
    2. ✅「関数名| 最適モデル(問題サイズ固定)| 最適モデル(コア数固定)」となるように表を操作
5. ✅「関数名｜モデル０｜．．．｜モデルN｜最適モデル」となる表をベンチマークCGで作成
6. ✅「5.」でやったことを他のベンチマークプログラムで実施

In [ ]:
# （～2022年1月14日）コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDict_fix_core = {}
resultDict_fix_class = {}

# resultDictAllWithoutInit = {}

for benchmarkName in benchmark_names:
    print(f"% _____benchmarkName={benchmarkName}_____\n\n\n")

    # 問題サイズを固定し、コア数を振る
    expVar_fix_class = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes_fix_class,
        processes=processes_fix_class,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )

    df_fix_class = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes_fix_class,
        targetClass=None,
        processes=processes_fix_class,
        targetProcess=None,
        expVar=expVar_fix_class,
        csvDirPath=csvDirPath,
    )
    df_fix_class = df_fix_class.rename(columns=dictColumnNamesJP)
    df_fix_class = df_fix_class.set_index("関数名")
    result_fix_class = addLowestMAPEsModelNameColumn(df_fix_class).rename(
        columns=dict_column_fix_class
    )
    resultDict_fix_class[benchmarkName.upper()] = result_fix_class

    # コア数を固定し、問題サイズを振る
    expVar_fix_core = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes_fix_core,
        processes=processes_fix_core,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )

    df_fix_core = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes_fix_core,
        targetClass=None,
        processes=processes_fix_core,
        targetProcess=None,
        expVar=expVar_fix_core,
        csvDirPath=csvDirPath,
    )
    df_fix_core = df_fix_core.rename(columns=dictColumnNamesJP)
    df_fix_core = df_fix_core.set_index("関数名")
    result_fix_core = addLowestMAPEsModelNameColumn(df_fix_core).rename(
        columns=dict_column_fix_core
    )
    resultDict_fix_core[benchmarkName.upper()] = result_fix_core

    # 作成した表を結合する
    merged_DF = pd.merge(df_fix_class, df_fix_core)
    dropped_column_name = merged_DF.columns.to_list()
    print(
        pd.merge(
            result_fix_class.drop(model_names_list_jp, axis=1),
            result_fix_core.drop(model_names_list_jp, axis=1),
            how="outer",
            right_index=True,
            left_index=True,
        ).to_latex(
            caption=f"ベンチマークプログラム{benchmarkName.upper()}における最適モデル",
            label=f"{date}bestModelCombinationIn{benchmarkName.upper()}",
        )
    )

In [ ]:
resultsDict = {}

# コア数を128に固定し、問題サイズを振る
classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128]
# ベンチマークプログラム名のリスト
# benchmark_names = ["cg", "ep", "ft", "is", "lu", "mg"]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月10日"

In [ ]:
# コア数と問題サイズの両方を振る

for benchmarkName in benchmark_names:
    print(f"% _____benchmarkName={benchmarkName}_____\n\n\n")

    # TODO: 1.「関数名｜モデル０｜．．．｜モデルN｜最適モデル」となる表をベンチマークCGで作成
    # TODO: 2.「1.」でやったことを他のベンチマークプログラムで実施

    expVar = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )
    print(f"expVar={expVar}")

    df = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVar,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )

    df = df.rename(columns=dictColumnNamesJP)
    df = df.set_index("関数名")
    result = addLowestMAPEsModelNameColumn(df)

    resultsDict[benchmarkName.upper()] = result

    print(
        result.to_latex(
            caption=f"ベンチマークプログラム{benchmarkName.upper()}における各モデルでのMAPE",
            label=f"{date}allModelsMAPEin{benchmarkName.upper()}",
        )
    )
#     print(
#         pd.merge(
#             result_fix_class.drop(model_names_list_jp, axis=1),
#             result_fix_core.drop(model_names_list_jp, axis=1),
#             how="outer",
#             right_index=True,
#             left_index=True,
#         ).to_latex(caption=f"ベンチマークプログラム{benchmarkName.upper()}における最適モデル", label="bestModelCombinationInCG")
#     )

# （コア数128＆異なる問題サイズでの結果）OR（異なるコア数＆異なる問題サイズでの結果）

In [ ]:
resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
date = "2021年11月17日"

resultDict128WithoutInit = {}
resultDict128WithInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(
    #     dfBase.to_latex(
    #         caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
    #         label=f"{date}BaseMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDict128WithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(
    #     dfInit.to_latex(
    #         caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
    #         label=f"{date}InitMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDict128WithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="128WithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="128WithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithoutInit).copy
)
#

# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(
    #     dfBase.to_latex(
    #         caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
    #         label=f"{date}BaseMAPE{benchmarkName}",
    #     )
    # )

    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(
    #     dfInit.to_latex(
    #         caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
    #         label=f"{date}InitMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")

# 異なるコア数＆異なる問題サイズでの結果

## 「コア数に線形モデル、問題サイズに反比例モデル」となる新モデルを追加

In [ ]:
resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
    "modelLinAndIp": "線形および反比例モデル",
}
model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
print(f"model_names_list={model_names_list}")
date = "2021年12月25日"

In [ ]:
# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(dfBase)
    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(dfInit)
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")

# 異なるコア数＆異なる問題サイズでの結果

## 「コア数に反比例モデル、問題サイズに線形モデル」となる新モデルを追加

In [ ]:
resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
    "modelIpAndLin": "反比例および線形モデル",
}
model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
print(f"model_names_list={model_names_list}")
date = "2022年1月3日"

In [ ]:
# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(dfBase)
    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(dfInit)
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")

# 異なるコア数＆異なる問題サイズでの結果

## 「コア数に線形モデル、問題サイズに対数モデル」となる新モデルを追加

In [ ]:
resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
    "modelLinAndLog": "線形および対数モデル",
}
model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
print(f"model_names_list={model_names_list}")
date = "2022年1月3日"

In [ ]:
# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(dfBase)
    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(dfInit)
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")

# 異なるコア数＆異なる問題サイズでの結果

## 「コア数に反比例モデル、問題サイズに対数モデル」となる新モデルを追加

In [ ]:
resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
    "modelIpAndLog": "反比例および対数モデル",
}
model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
print(f"model_names_list={model_names_list}")
date = "2022年1月3日"

In [ ]:
# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(dfBase)
    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(dfInit)
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")

# 異なるコア数＆異なる問題サイズでの結果

## 「コア数に対数モデル、問題サイズに線形モデル」となる新モデルを追加

In [ ]:
resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
    "modelLogAndLin": "対数および線形モデル",
}
model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
print(f"model_names_list={model_names_list}")
date = "2022年1月3日"

In [ ]:
# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(dfBase)
    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(dfInit)
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")

# 異なるコア数＆異なる問題サイズでの結果

## 「コア数に対数モデル、問題サイズに線形モデル」となる新モデルを追加

In [ ]:
resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
    "modelLogAndIp": "対数および反比例モデル",
}
model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
print(f"model_names_list={model_names_list}")
date = "2022年1月3日"

In [ ]:
# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(dfBase)
    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(dfInit)
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")